# PyTorch 3D ResNet

In [1]:
import torch
from tqdm import tqdm
from time import time
from sklearn.metrics import accuracy_score, r2_score, precision_score, recall_score, f1_score
import pandas as pd

from torch.utils.data import DataLoader, Subset, random_split

from models.ThreeDResNet import get_3dResNet, get_resnet_transformer
from colorVideoDataset import ColorVideoDataset

c:\Users\Admin\Documents\ConvLSTMPyTorchLightning\.venv\Lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\Admin\Documents\ConvLSTMPyTorchLightning\.venv\Lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
MODEL = get_3dResNet()
DATASET = ColorVideoDataset('./colors')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Using cache found in C:\Users\Admin/.cache\torch\hub\facebookresearch_pytorchvideo_main


Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
      (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(np.int64(1), np.int64(1), np.int64(1)), bias=False)
          (branch1_norm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (norm_a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(64, 64, kern

Train, Test, Val Sets

In [3]:
train_dataset, test_dataset = random_split(DATASET, [int(0.8 * len(DATASET)), len(DATASET) - int(0.8 * len(DATASET))])
test_dataset, val_dataset = random_split(test_dataset, [int(0.5 * len(test_dataset)), len(test_dataset) - int(0.5 * len(test_dataset))])

In [4]:
def get_dataloader(dataset, subset_ratio : float | None = 0.1, batch_size : int = 2):
    transform = get_resnet_transformer()

    def collate_fn(batch):
        videos = []
        labels = []
        for video, label, _ in batch:
            video = video.permute(1, 0, 2, 3)
            video = transform({"video": video})["video"]
            videos.append(video)
            labels.append(torch.tensor(label, dtype=torch.long))
        
        videos = torch.stack(videos)
        labels = torch.stack(labels)
        return videos, labels
    
    if subset_ratio is not None:
        num_samples = int(len(dataset) * subset_ratio) 
        subset_indices = list(range(num_samples))
        subset = Subset(dataset, subset_indices)
        dataloader = DataLoader(subset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    else:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    return dataloader

In [5]:
def infer_loop(model, dataloader, criterion) -> tuple:
    running_loss = 0.0
    all_preds = []
    all_labels = []
    model.eval()
    for videos, labels in dataloader:
        videos, labels = videos.to(DEVICE), labels.to(DEVICE)

        outputs = model(videos)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        all_preds.extend(outputs.argmax(dim=1).cpu().detach().numpy())
        all_labels.extend(labels.cpu().detach().numpy())
        
    val_accuracy = accuracy_score(all_preds, all_labels)
    avg_loss = running_loss / len(dataloader)

    return (val_accuracy, avg_loss)

In [ ]:
def training_loop(model, train_dataloader, val_dataloader : DataLoader | None = None, epochs=5, learning_rate=1e-4):
    results_dict = {
        'time_per_batch': [],
        'time_per_epoch': [],
        'train_accuracy': [],
        'train_loss': [],
        'val_accuracy': [],
        'val_loss': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'r2': [],
    }
    model = model.to(DEVICE)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    epoch_pbar = tqdm(range(epochs), desc="Training Epochs")

    model.train()
    for epoch in epoch_pbar:
        running_loss = 0.0
        epoch_start = time()
        all_preds = []
        all_labels = []
        
        for videos, labels in train_dataloader:
            videos, labels = videos.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(videos)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
            # Store predictions and labels for metrics
            all_preds.extend(outputs.argmax(dim=1).cpu().detach().numpy())
            all_labels.extend(labels.cpu().detach().numpy())
        
        epoch_time = time() - epoch_start
        results_dict['time_per_epoch'].append(epoch_time)
        
        # Calculate metrics
        results_dict['train_accuracy'].append(accuracy_score(all_labels, all_preds))
        results_dict['precision'].append(precision_score(all_labels, all_preds, average='weighted', zero_division=0))
        results_dict['recall'].append(recall_score(all_labels, all_preds, average='weighted', zero_division=0))
        results_dict['f1'].append(f1_score(all_labels, all_preds, average='weighted', zero_division=0))
        results_dict['r2'].append(r2_score(all_labels, all_preds))
        
        avg_loss = running_loss / len(train_dataloader)
        results_dict['train_loss'].append(avg_loss)
        epoch_pbar.set_postfix(avg_loss=f"{avg_loss:.4f}")

    val_acc, val_loss = infer_loop(model, val_dataloader, criterion)
    results_dict['val_accuracy'].append(val_acc)
    results_dict['val_loss'].append(val_loss)
    
    return results_dict

In [7]:
dataloader = get_dataloader(DATASET, subset_ratio=0.1, batch_size=16)
x, y = next(iter(dataloader))
print(x.shape, y.shape)
del x, y

torch.Size([16, 3, 8, 256, 256]) torch.Size([16])


In [8]:
train_dataloader =  get_dataloader(train_dataset, subset_ratio=None, batch_size=4)
val_dataloader =  get_dataloader(val_dataset, subset_ratio=None, batch_size=4)
test_dataloader =  get_dataloader(test_dataset, subset_ratio=None, batch_size=4)

In [9]:
results = training_loop(MODEL, train_dataloader, val_dataloader, epochs=100, learning_rate=2e-4)

Training Epochs:   1%|          | 1/100 [00:52<1:26:46, 52.59s/it]

Epoch [1/100], Loss: 2.0465


Training Epochs:   2%|▏         | 2/100 [01:40<1:21:47, 50.07s/it]

Epoch [2/100], Loss: 1.8105


Training Epochs:   3%|▎         | 3/100 [02:30<1:20:21, 49.71s/it]

...

Epoch [39/100], Loss: 0.3576


Training Epochs:  40%|████      | 40/100 [29:52<43:45, 43.76s/it]

Epoch [40/100], Loss: 0.3088


Training Epochs:  40%|████      | 40/100 [30:23<45:35, 45.60s/it]


KeyboardInterrupt: 

In [ ]:
results

{'time_per_batch': [0.26043272018432617,
  0.04736924171447754,
  0.051451921463012695,
  0.0431818962097168,
  0.04414844512939453,
  0.05073857307434082,
  0.04169797897338867,
  0.03714585304260254,
  0.06663298606872559,
  0.0694270133972168,
  0.0866551399230957,
  0.0837092399597168,
  0.08386969566345215,
  0.08619928359985352,
  0.07990288734436035,
  0.07819128036499023,
  0.0991065502166748,
  0.12497091293334961,
  0.09679746627807617,
  0.2366929054260254,
  0.051824331283569336,
  0.05388641357421875,
  0.03834795951843262,
  0.08256721496582031,
  0.07532191276550293,
  0.09441947937011719,
  0.08146357536315918,
  0.07831835746765137,
  0.08380961418151855,
  0.06888031959533691,
  0.08284783363342285,
  0.07780265808105469,
  0.08165574073791504,
  0.09861326217651367,
  0.1150963306427002,
  0.09218430519104004,
  0.08516550064086914,
  0.08716821670532227,
  0.08283019065856934,
  0.1100468635559082,
  0.0975348949432373,
  0.08039689064025879,
  0.16640973091125488,


In [ ]:
results_df = pd.DataFrame.from_dict(results, orient="index").T
results_df.to_csv("3D_ResNet_Results.csv")